# Simulation processing

Here we have some examples about how to process a simulation and save it in the simulation folder ready for optimisation. We have to run the code in the cell below to allow the paths to work in this notebook.

In [1]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

/home/dharrington/cyclops/src


## Reading a 2D scalar field from a simulation


In this code we have a 3D simulation file ('monoblock_out.e') which has one face of its surface labelled 'right'. We read this file and create a 2D scalar field from it's relevant face, plus a grid of comparision points for the optimisation loss function.

Change:
1. Simulation file name `monoblock_out.e`
2. Sensor region from `right`
3. Variable name from `temperature`
4. Names of the grid and field from `test_scalar_field.obj` and `test_grid.obj`

In [2]:
from sim_reader import *


# Define the necessary objects
reader = ExodusReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
grid = unfolder.generate_grid(bounds, 30, 30)

temps = reader.read_scalar(sensor_region, 'temperature')
temp_field = ScalarField(LModel, bounds, 2)
temp_field.fit_model(pos_2D, temps)

# Save the simulation data
pickle_manager.save_file('simulation', 'field_temp.obj', temp_field)
pickle_manager.save_file('simulation', 'grid_temp.obj', grid)

<meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress


## Reading a 2D vector field from a simulation

We do the same as above, except for a vector field not a scalar field. Note that the ExodusReader will break any vectors into their components when it reads them, so you should look in the code above to check what your components have been renamed as. For the 'monoblock_out.e' case the disp vector has been broken down into `disp_x`, `disp_y` and `disp_z`.

In [3]:
from sim_reader import *


# Define the necessary objects
reader = ExodusReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
grid = unfolder.generate_grid(bounds, 30, 30)

disp = np.array([
    reader.read_scalar(sensor_region, 'disp_x'),
    reader.read_scalar(sensor_region, 'disp_y'),
    reader.read_scalar(sensor_region, 'disp_z')
]).T

disp_field = VectorField(LModel, bounds, 2)
disp_field.fit_model(pos_2D, disp)

# Save the simulation data
pickle_manager.save_file('simulation', 'field_disp.obj', disp_field)
pickle_manager.save_file('simulation', 'grid_disp.obj', grid)

<meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress


## Reading a 1D scalar line from a field

Of course you can read in a 1D scalar line directly from a simulation file, but in our case we don't have a 1D scalar line simulation file. However, we know that the Monoblock field is uniform on the 'right' face so we can compress it to a line to speed up the optimisation by reducing the search space. Note that the last number in the ScalarField constructor has been changed to 1 as the line-field is only 1 dimensional.

In [4]:
from sim_reader import *

# Define the necessary objects
reader = ExodusReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
temps = reader.read_scalar(sensor_region, 'temperature')

# Define the 2D field objects
temp_field = ScalarField(LModel, bounds, 2)
temp_field.fit_model(pos_2D, temps)

# Convert the 2D field to a 1D line
pos1 = (bounds[0][0], bounds[0][1])
pos2 = (bounds[0][0], bounds[1][1])
line_2D = unfolder.generate_line(pos1, pos2, 50)
line_temps = temp_field.predict_values(line_2D)
line_1D = unfolder.compress_1D(line_2D)

bounds_1D = [line_1D[0], line_1D[-1]]
new_line_field = ScalarField(CSModel, bounds_1D, 1)
new_line_field.fit_model(line_1D, line_temps)

# Save the new 1D line field
pickle_manager.save_file('simulation', 'field_temp_line.obj', new_line_field)
pickle_manager.save_file('simulation', 'line_temp.obj', line_1D)

<meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress


## Reading a vector line from a field

We'll do the same thing only for displacement.

In [5]:
from sim_reader import *

# Define the necessary objects
reader = ExodusReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)

disp = np.array([
    reader.read_scalar(sensor_region, 'disp_x'),
    reader.read_scalar(sensor_region, 'disp_y'),
    reader.read_scalar(sensor_region, 'disp_z')
]).T

disp_field = VectorField(LModel, bounds, 2)
disp_field.fit_model(pos_2D, disp)

# Convert the 2D field to a 1D line
pos1 = (bounds[0][0], bounds[0][1])
pos2 = (bounds[0][0], bounds[1][1])
line_2D = unfolder.generate_line(pos1, pos2, 50)
line_disps = disp_field.predict_values(line_2D)
line_1D = unfolder.compress_1D(line_2D)

bounds_1D = [line_1D[0], line_1D[-1]]
new_line_field = VectorField(CSModel, bounds_1D, 1)
new_line_field.fit_model(line_1D, line_disps)

# Save the new 1D line field
pickle_manager.save_file('simulation', 'field_disp_line.obj', new_line_field)
pickle_manager.save_file('simulation', 'line_disp.obj', line_1D)

<meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress
